In [1]:
import os
import json
import mlflow

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import OpenSearchVectorSearch
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_aws import BedrockLLM

import sagemaker
import boto3

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/willem/.config/sagemaker/config.yaml


In [2]:
mlflow.set_tracking_uri("http://localhost:5000")

In [3]:
session = boto3.Session(profile_name='vrt-analytics-engineer-nonsensitive')
sagemaker_session = sagemaker.Session(boto_session=session)
role = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)

In [4]:
# 1. Get retriever
# embed stuff and write to opensearch
model_name = "NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers"
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)
# Init OpenSearch client connection
docsearch = OpenSearchVectorSearch(
     index_name="aoss-index",  # TODO: use the same index-name used in the ingestion script
     embedding_function=embeddings,
     opensearch_url="https://epcavlvwitam2ivpwv4k.eu-west-2.aoss.amazonaws.com:443",  # TODO: e.g. use the AWS OpenSearch domain instantiated previously
#     http_auth=("<<insert_user_name>>", "<<insert_password>>"),
#     use_ssl = False,
#     verify_certs = False,
#     ssl_assert_hostname = False,
#     ssl_show_warn = False,
)

/home/willem/miniconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/willem/miniconda3/envs/llm/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
def loader_fn():
    model_name = "NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers"
    encode_kwargs = {'normalize_embeddings': False}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        encode_kwargs=encode_kwargs
    )
    # Init OpenSearch client connection
    docsearch = OpenSearchVectorSearch(
        index_name="aoss-index",  # TODO: use the same index-name used in the ingestion script
        embedding_function=embeddings,
        opensearch_url="https://epcavlvwitam2ivpwv4k.eu-west-2.aoss.amazonaws.com:443",  # TODO: e.g. use the AWS OpenSearch domain instantiated previously
    #     http_auth=("<<insert_user_name>>", "<<insert_password>>"),
    #     use_ssl = False,
    #     verify_certs = False,
    #     ssl_assert_hostname = False,
    #     ssl_show_warn = False,
    )

    return docsearch.as_retriever()

In [5]:
# 2. Get LLM
llm = BedrockLLM(
    credentials_profile_name="vrt-analytics-engineer-nonsensitive", 
    region_name="eu-west-2",
    model_id="meta.llama3-70b-instruct-v1:0"
)

In [8]:
# 3. Create model in MLFLOW
mlflow.set_experiment("VRT MAX RAG v1")

retrievalQA = RetrievalQA.from_llm(llm=llm, retriever=docsearch.as_retriever())

persist_dir = "../data/faiss"
with mlflow.start_run() as run:
    model_info = mlflow.langchain.log_model(
        retrievalQA,
        artifact_path="rag",
        loader_fn=loader_fn
    )

2024/08/06 20:35:37 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpj8ekv8f0/model, flavor: langchain). Fall back to return ['langchain==0.2.8', 'pydantic==2.8.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
/home/willem/miniconda3/envs/llm/lib/python3.10/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
